IMPORT

In [ ]:
import pandas as pd
import csv
import random 
import numpy as np


FUNCTIONS

FUNCTION LEGGIDATI

In [ ]:

def LeggiDati(filename):

    df=pd.read_csv(filename,header=0)

    corse=[]
    fermate=df.columns[2:].tolist()  #creo una lista con le fermate prendendo dalla seconda colonna in avanti

    for _,riga in df.iterrows():  #iterrows itera per indice e contenuto della riga, con i nomi delle colonne come chiavi.
        autista=riga.iloc[0]
        passeggeri=[]
        for dato in riga[2:]:
            if pd.isna(dato) or dato == '--': # il comando .isna() serve per capire se il valore è nullo
                passeggeri.append((0, 0))
            elif isinstance(dato, str) and '/' in dato: # il comando isintance(dato,str) controlla se dato è una stringa-> ritorna vero o falso
                saliti, scesi = map(int, dato.split('/')) # il comando map(funzione,iterabile) serve per applicare una funzione a ogni elemento di un iterabile
                passeggeri.append((saliti, scesi)) 
            elif isinstance(dato, str) and dato.startswith('-'):
                scesi = int(dato.rstrip('-'))
                passeggeri.append((0, scesi))
            else:
                    try:
                        saliti = int(dato)
                        passeggeri.append((saliti, 0))
                    except:
                        passeggeri.append((0, 0))

        corsa = {'autista': autista, 'passeggeri': passeggeri}
        corse.append(corsa)
    
    return df,corse,fermate




#for corsa in corse:
   # print(corsa)

FUNCTION ESTRAI_SALITI_SCESI

In [ ]:
def estrai_saliti_scesi(val):
    if val == '--' or pd.isna(val):
        return (0, 0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (0, -int(val.strip('-')))
    else:
        try:
            return (int(val), 0)
        except:
            return (0, 0)

FUNCTION ESTRAI_SALITI_SCESI_CLA

In [ ]:
def estrai_saliti_scesi_cla(val):
    if val == '--' or pd.isna(val):
        return ('0/0', 0,0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (val,int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (f"0/{int(val.strip('-'))}",0, -int(val.strip('-')))
    else:
        try:
            return (f'{int(val)}/0',int(val), 0)
        except:
            return ('0/0',0, 0)

FUNCTION LEGGINOMI

In [ ]:
def LeggiNomi(filename): 
    df=pd.read_csv(filename, sep=':',header=None, names=['Code','Name'])
    dict_nomi={}
    for _,riga in df.iterrows():
        dict_nomi[riga.iloc[0]]=riga.iloc[1]
    #print(dict_nomi)

    return df, dict_nomi

GET DATA

In [ ]:
df_db= LeggiNomi('database.txt')[0]
df_db2=LeggiNomi('database.txt')[0]
df_ae=LeggiDati('dati_aeroporto_torino.csv')[0]
# i read.csv sono dentro le funzioni

GET TOTAL NUMBER OF PASSENGERS

In [ ]:

df_ae_melt=pd.melt(df_ae,id_vars=['ID', 'Time'],var_name='Fermata_ID',value_name='Passeggeri')  #idvars =identificativo di oggni riga    varname=come chiamo la nuova colonna che prende i nomi delle colonne passate(eccetto quelle identificative) 
df_ae_melt.rename(columns={'ID':'Autista_ID'}, inplace=True)

df_ae_melt['Passeggeri2'] = df_ae_melt['Passeggeri'].astype(str).str.strip() #rimuovo spazi e mi assicuro che siano stringhe, così le converto con la prossima funzione in interi
df_ae_melt[['Passeggeri2','Saliti', 'Scesi']] = df_ae_melt['Passeggeri2'].apply(estrai_saliti_scesi_cla).apply(pd.Series) #applico la funzione estrai_saliti_scesi a tutte le righe

df_ae_melt['Totale_Passeggeri']=df_ae_melt['Saliti'] - (df_ae_melt['Scesi'])
display(df_ae_melt.tail(10))


GET DF_DB_DRIVERS AND DF_DB_STOPS OUT OF DF_DB

In [ ]:
df_db['Tipo'] = np.where(df_db['Code'].str.contains('A'), 'autista', 'fermata')
#display(df_db)


df_db_drivers= df_db.loc[df_db['Tipo']=='autista']
df_db_drivers.rename(columns={'Code':'Autista_ID'}, inplace=True)
df_db_stops=df_db.loc[df_db['Tipo']=='fermata']
df_db_stops.rename(columns={'Code':'Fermata_ID'}, inplace=True)
display(df_db_drivers)
display(df_db_stops)

ASSOCIATED DF_DB_DRIVERS AND DF_AE_MELT

In [ ]:

#PRIMO JOIN
df_joined=df_ae_melt.merge(df_db_drivers, how='left', on='Autista_ID')                                                          #nella riga in basso: non ho messo la colonna tipo, nel dataframe joined
df_joined=df_joined[['Autista_ID','Name','Time','Fermata_ID','Passeggeri','Passeggeri2','Saliti','Scesi','Totale_Passeggeri']] #ho spostato la colonna 'Name' affianco a quella di 'Autista_ID'
df_joined.rename(columns={'Name':'Autista_Name'}, inplace=True)
#display(df_joined.sort_values(by='Totale_Passeggeri'))
display(df_joined)
#display(df_joined.iloc[50:100])



ASSOCIATED DF_JOINED AND DF_DB_STOPS

In [ ]:
'''SECONDO JOIN'''
df_joined2=df_joined.merge(df_db_stops, on='Fermata_ID')
df_joined2.rename(columns={'Name':'Fermata_Name'}, inplace=True)
df_joined2=df_joined2[['Autista_ID','Autista_Name','Time','Fermata_ID','Fermata_Name','Passeggeri','Passeggeri2','Saliti','Scesi','Totale_Passeggeri']]
display(df_joined2)

GET MAX/MIN PASSENGERS

In [ ]:
'''RACCOLGO SALITI E SCESI'''
SalitiRaccolti=df_joined2.groupby('Fermata_Name')['Saliti'].sum()
#display(SalitiRaccolti)
ScesiRaccolti=df_joined2.groupby('Fermata_Name')['Scesi'].sum()
#display(ScesiRaccolti)

max_saliti=SalitiRaccolti.max()
#display(max_saliti)
max_scesi=-(ScesiRaccolti.min())

fermata_max= SalitiRaccolti.idxmax()
#display(indice_fermata_max)
fermata_min=ScesiRaccolti.idxmin()

GET SOLUTION

In [ ]:


'''PRINT E SOLUZIONE'''
#MaxSaliti=df_fermateRaccolte['Saliti'].max()
#display(MaxSaliti)
print(f'La fermata con più saliti è {fermata_max} con {max_saliti} passeggeri saliti ')
print(f'La fermata con più scesi è {fermata_min} con {max_scesi} passeggeri scesi')

#print(df_joined.groupby('Autista_ID')['Saliti'].sum())
SalitiXAutista=df_joined.groupby('Autista_ID')['Saliti'].sum().to_dict() #creo dizionario con chiave autista id e valore la somma dei saliti per quell'autista id
#print(SalitiXAutista)

SalitiXAutistaOrdinato= dict(sorted(SalitiXAutista.items(), key=lambda item: item[1], reverse=True)) #ordino il dizionario in modo decrescente
print(SalitiXAutistaOrdinato)
print('Lista di autisti ordinata in base al numero di passeggeri saliti in una giornata:')

for Autista_ID in SalitiXAutistaOrdinato:
    print(f'{Autista_ID} con {SalitiXAutistaOrdinato[Autista_ID]} passeggeri')

#devo ricordarmi di cancellare l'output prima del commit!!
